<a href="https://colab.research.google.com/github/aakanshadalmia/Data-Science/blob/main/Credit_Card_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Credit Card Fraud Detection
-------------------------

**Problem Statement:** The challenge is for credit card companies to recognize fraudulent credit card transactions so that customers are not charged for items that they did not purchase.
 
**Goal:** This project is aimed at finding a classification model with the best performance for an imbalanced dataset.

**Evaluation Metrics Used:** AUC-ROC Curve

**To be Explored**:
- Hyperparamter tuning

- PCA

- Decision tree Visualization


## About the Dataset
---------

The dataset contains transactions made by credit cards in September 2013 by European cardholders.

This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is **highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.**

It contains only numerical input variables which are the result of a PCA transformation. Due to confidentiality issues, the original features and more background information about the data has been omitted.

Features V1, V2, … V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-sensitive learning. 

**Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.**

_Given the class imbalance ratio, Area Under the Precision-Recall Curve (AUPRC) has been used for evaluation instead of using a Confusion matrix to determine accuracy for the unbalanced classification._

In [21]:
#Import all required libraries

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA

In [6]:
#Reading data from csv file into a dataframe

data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/creditcard.csv")

In [7]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [8]:
data.isna().sum()
data = data.fillna(0)

In [9]:
#Estimating the imbalance in the dataset

print("Total number of transactions:", len(data))
normal = len(data[data.Class == 0])
fraud = len(data[data.Class == 1])
print("Legitimate Transactions:", normal)
print("Fraud Transactions:", fraud)
print("% of Fraud Transactions:", round(fraud / len(data) * 100,4))

Total number of transactions: 284807
Legitimate Transactions: 284315
Fraud Transactions: 492
% of Fraud Transactions: 0.1727


In [10]:
#Defining the training and testing datasets

X = data.drop('Class',axis = 1)
Y = data.Class.values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 1)

# Principal Component Analysis

In [29]:
pca = PCA(n_components = 10).fit(X_train)

In [30]:
X_train_pca = pca.transform(X_train)
X_train_pca.shape

(199364, 10)

In [31]:
X_test_pca = pca.transform(X_test)


## K-Nearest Neighbours

In [11]:
#Model Building

kNN = KNeighborsClassifier(n_neighbors = 5, n_jobs = -1, leaf_size = 600)
kNN.fit(X_train, Y_train)

#Applying model to test dataset
kNN_Y_pred = kNN.predict(X_test)

In [70]:
#Model Evaluation

kNN_f1 = metrics.f1_score(Y_test,kNN_Y_pred)
print("F1 score for kNN model is:",kNN_f1)

F1 score for kNN model is: 0.05755395683453238


In [60]:
#Hyperparameter Tuning
grid_params = {'n_neighbors' : [3,9],
               'metric' : ['euclidean','manhattan']}

gridsearch_kNN = GridSearchCV(KNeighborsClassifier(), grid_params, cv = 3, scoring = 'f1', n_jobs = -1, verbose = 10)
gridsearch_kNN.fit(X_train_pca, Y_train)
gridsearch_kNN

Fitting 3 folds for each of 4 candidates, totalling 12 fits


GridSearchCV(cv=3, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'metric': ['euclidean', 'manhattan'],
                         'n_neighbors': [3, 9]},
             scoring='f1', verbose=10)

In [61]:
print("Best possible Cross Validation score:",gridsearch_kNN.best_score_)

Best possible Cross Validation score: 0.25682148264886745


In [62]:
gridsearch_kNN.best_params_

{'metric': 'manhattan', 'n_neighbors': 3}

In [68]:
kNN_tuned = KNeighborsClassifier(gridsearch_kNN.best_params_['n_neighbors'], metric = gridsearch_kNN.best_params_['metric'])
kNN_tuned.fit(X_train_pca,Y_train)
kNN_tuned_Y_pred = kNN_tuned.predict(X_test_pca)

In [67]:
#Model Evaluation

kNN_best_f1 = metrics.f1_score(Y_test,kNN_tuned_Y_pred)
print("F1 score for kNN model after hyperparameter tuning is:",kNN_best_f1)

F1 score for kNN model after hyperparameter tuning is: 0.18791946308724833


## Logistic Regression

In [10]:
#Model Building

log_reg = LogisticRegression(verbose = 2, n_jobs = -1)
log_reg.fit(X_train, Y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   20.3s finished


LogisticRegression(n_jobs=-1, verbose=2)

In [11]:
#Applying model to test dataset

LR_Y_pred = log_reg.predict(X_test)

In [12]:
#Model Evaluation

fpr, tpr, thresholds = metrics.roc_curve(Y_test,LR_Y_pred)
lr_auc = metrics.auc(fpr,tpr)
print("AUC for Logistic Regression model is:",lr_auc)

AUC for Logistic Regression model is: 0.8071964072667406


## Decision Tree

In [13]:
#Model Building

DT = DecisionTreeClassifier(criterion = 'entropy')
DT.fit(X_train,Y_train)

DecisionTreeClassifier(criterion='entropy')

In [14]:
#Applying model to test dataset

DT_Y_pred = DT.predict(X_test)

In [15]:
#Model Evaluation

fpr, tpr, thresholds = metrics.roc_curve(Y_test,DT_Y_pred)
dt_auc = metrics.auc(fpr, tpr)
print("AUC for Decision Tree model is:",dt_auc)

AUC for Decision Tree model is: 0.866479110986074


## Support Vector Machine

In [16]:
#Model Building

svm = SVC()
svm.fit(X_train, Y_train)

SVC()

In [17]:
#Applying model to test dataset

svm_Y_pred = svm.predict(X_test)

In [18]:
#Model Evaluation

fpr, tpr, thresholds = metrics.roc_curve(Y_test,svm_Y_pred)
svm_auc = metrics.auc(fpr, tpr)
print("AUC for SVM model is:",svm_auc)

AUC for SVM model is: 0.5


## Random Forest

In [19]:
#Model Building

rf = RandomForestClassifier(n_jobs = -1)
rf.fit(X_train, Y_train)

RandomForestClassifier(n_jobs=-1)

In [20]:
#Applying model to test dataset

rf_Y_pred = rf.predict(X_test)

In [21]:
#Model Evaluation

fpr, tpr, thresholds = metrics.roc_curve(Y_test,rf_Y_pred)
rf_auc = metrics.auc(fpr, tpr)
print("AUC for Random Forest model is:",rf_auc)

AUC for Random Forest model is: 0.8888419999687407


## XGBoost

In [22]:
#Model Building

xgb =XGBClassifier()
xgb.fit(X_train, Y_train)

XGBClassifier()

In [23]:
#Applying model to test dataset

xgb_Y_pred = rf.predict(X_test)

In [24]:
#Model Evaluation

fpr, tpr, thresholds = metrics.roc_curve(Y_test,xgb_Y_pred)
xgb_auc = metrics.auc(fpr, tpr)
print("AUC for XGBoost model is:",xgb_auc)

AUC for XGBoost model is: 0.8888419999687407


# Comparison of Model Performance

In [25]:
df_perf = pd.DataFrame(data = [['KNN',knn_auc],['Logistic Regression',lr_auc],
                               ['Decision Tree',dt_auc],['Support Vector Machine',svm_auc],
                               ['Random Forest',rf_auc],['XGBoost',xgb_auc]],
                       columns = ['Model','AUC'])
df = df_perf.sort_values(by = 'AUC')
df

,Model,AUC
3,Support Vector Machine,0.500000
0,KNN,0.514815
1,Logistic Regression,0.807196
2,Decision Tree,0.866479
4,Random Forest,0.888842
5,XGBoost,0.888842
